In [ ]:
%%timeit -n 10 -r 30

import pymysql

class Database:
    def __init__(self, host, port, user, password, database):
        self.connection = pymysql.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=database,
            cursorclass=pymysql.cursors.DictCursor,
            use_unicode=True,
            charset="utf8mb4",
        )

    def execute_query(self, query):
        with self.connection.cursor() as cursor:
            cursor.execute(query)
            return cursor.fetchall()

    def close(self):
        self.connection.close()


def query_prodotti_ordinati(database, data_inizio, data_fine):
    query = """
        SELECT 
            po.ID AS ProdottoOrdinatoID,
            po.NumeroOrdine,
            po.NomeModello,
            po.Quantita,
            po.PrezzoUnitario AS PrezzoUnitarioProdotto,
            po.Valuta AS ValutaProdotto,
            po.DataAggiunta,
            po.DataConsegnaPrevista,
            po.StatoProdotto,
            po.NumeroRMA,
            po.Note AS NoteProdotto,
            ae.NomeModello AS NomeModelloApparecchiatura,
            ae.NumeroSerie,
            ae.DataProduzione,
            ae.Reparto,
            ae.Fornitore AS FornitoreApparecchiatura,
            ae.DataAcquisto,
            ae.Cliente,
            ae.RepartoAssistenza,
            ae.DataUltimaManutenzione,
            ae.GaranziaScadenza,
            ae.DescrizioneProblema,
            ae.ComponenteDifettoso,
            ae.TecnicoAssistenza,
            ae.Note AS NoteApparecchiatura,
            oc.NumeroOrdine AS NumeroOrdineCliente,
            oc.DataOrdine,
            oc.Cliente AS ClienteOrdine,
            oc.IndirizzoSpedizione,
            oc.CittaSpedizione,
            oc.CAPSpedizione,
            oc.ProvinciaSpedizione,
            oc.NazioneSpedizione,
            oc.MetodoPagamento,
            oc.StatoPagamento,
            oc.DataPagamento,
            oc.MetodoSpedizione,
            oc.DataSpedizione,
            oc.StatoSpedizione,
            oc.DataConsegna,
            oc.TotaleOrdine,
            oc.Valuta AS ValutaOrdine,
            oc.NumeroFattura,
            oc.CodicePromozionale,
            oc.Note AS NoteOrdine
        FROM ProdottiOrdinati po
        JOIN ApparecchiatureElettroniche ae ON po.NomeModello = ae.NomeModello
        JOIN OrdiniClienti oc ON po.NumeroOrdine = oc.NumeroOrdine
        WHERE po.DataAggiunta BETWEEN %s AND %s
            AND ae.DataProduzione <= %s
            AND oc.DataOrdine >= %s;
    """

    return database.execute_query(query, (data_inizio, data_fine, data_fine, data_inizio))


if __name__ == "__main__":
    database = Database(
        host="localhost",
        port=3306,
        user="root",
        password="password",
        database="my_database",
    )

    data_inizio = "2022-01-01"
    data_fine = "2023-06-30"

    output = query_prodotti_ordinati(database, data_inizio, data_fine)

    print("Output della funzione di query:")
    print(output)
    print("Numero di righe selezionate:", len(output))
    print("Colonne prese in considerazione:", list(output[0].keys()))

    database.close()

